### Some analysis

In [1]:
from officelib.xllib import *
from officelib.const import xlconst as xlc
import os

In [2]:
keys = {
    'temperature.pv',
    'agitation.pv',
    'ph.outputDown',
    'ph.manDown',
    'do.manUp',
    'do.outputDown',
    'do.pv',
    'do.manDown',
    'do.outputUp',
    'maingas.man',
    'maingas.mode',
    'maingas.pv',
    'MFCs.air',
    'MFCs.n2',
    'MFCs.o2',
    'MFCs.co2',
    'level.pv',
    'Elapsed',
    'Elapsed.hr',
    'Time'
}
def hide_columns(ws):
    for col in ws.UsedRange.Columns:
        c = col.Cells(1,1)
        if c.Value not in keys:
            col.Hidden=True

In [3]:
def data_range(ws, name, begin_offset=0):
    c1 = ws.Cells.Find(name)
    if c1 is None: raise ValueError(name)
    c1 = c1.Offset(2+begin_offset,1)
    c2 = c1.End(xlc.xlDown)
    return c1, c2

class SeriesProxy():
    def __init__(self, ws, name, xname="Elapsed", yax=1, begin_offset=0):
        self.ws = ws
        self.name = name
        self.yax = yax
        self.s = None
        self.xname = xname
        self.begin_offset=begin_offset
        
    def create(self, chart):
        e1, e2 = data_range(self.ws, self.xname, self.begin_offset)
        c1, c2 = data_range(self.ws, self.name, self.begin_offset)
        cr = self.ws.Cells.Range
        self.xr = cr(e1, e2)
        self.yr = cr(c1, c2)
        self.s = CreateDataSeries(chart, self.xr, self.yr, self.name)
        
    def set_name(self, n):
        self.s.Name = n 
        
class ChartProxy():
    def __init__(self, ws, chart=None, *args):
        self.ws = ws
        self.chart = chart or CreateChart(ws, *args)
        self.chart.ChartStyle = 240  # Excel 2017 style
        self.series = []
        
    def make_series(self, series):
        for s in series:
            s.create(self.chart)
            self.series.append(s)
            
    def _axlabel(self, ax, t):
        ax.HasTitle=True
        ax.AxisTitle.Text=t
    
    def xlabel(self, t):
        xax = self.chart.Axes(xlc.xlCategory, xlc.xlPrimary)
        self._axlabel(xax, t)
        
    def ylabel(self, t):
        yax = self.chart.Axes(xlc.xlValue, xlc.xlPrimary)
        self._axlabel(yax, t)
        
    def ylabel2(self, t):
        yax = self.chart.Axes(xlc.xlValue, xlc.xlSecondary)
        self._axlabel(yax, t)
        
    def title(self, t):
        self.chart.HasTitle = True
        self.chart.ChartTitle.Text = t
        
    def copyto(self, r):
        # chart parent -> ChartObject
        self.chart.Parent.Copy()
        r.Paste()
         
def _listify(args):
    l = []
    if isinstance(args, str):
        return [args]
    for a in args:
        if isinstance(a, (list,tuple)):
            l.extend(_listify(a))
        else:
            l.append(a)
    return l
    
def make_chart(names, title=None, xlabel=None, ylabel=None, xname="Elapsed", *, uws=None, begin_offset=0):
    uws = uws or ws
    c = ChartProxy(uws)
        
    names = _listify(names)
    c.make_series(SeriesProxy(uws,n, xname, 1, begin_offset) for n in names)
    
    # optional setup
    if title: c.title(title)
    if xlabel: c.xlabel(xlabel)
    if ylabel: c.ylabel(ylabel)
    
    return c

### Example code for now to use functions

In [4]:
# # MFCs
# mfcc = [
#     "MFCs.Air",
#     "MFCs.O2",
#     "MFCs.N2",
#     "MFCs.CO2"
# ]
# make_chart(mfcc, "MFC Flow", "Time(s)", "Flow Rate (LPM)")
# make_chart("do.pv", "DO PV", "Time(s)", "DO PV (%)")

#### Some analysis

In [24]:
def column_data(name):
    c1 = ws.Cells.Find(name)
    c2 = c1.End(xlc.xlDown)
    data = ws.Cells.Range(c1.Offset(2,1),c2).Value
    return [d[0] for d in data]
    
def add_af1(name, new, form):
    e1 = ws.Cells.Find(name)
    e1.Offset(1,2).EntireColumn.Insert()
    e1.Offset(1,2).Value = new
    
    c1 = e1.Offset(2,2)
    c2 = e1.End(xlc.xlDown).Offset(1,2)
    c1.Value = form % e1.Offset(2,1).GetAddress(0,0)
    afr = ws.Cells.Range(c1, c2)
    c1.AutoFill(afr)
    return c1, c2
    

def nch(ws, n):
    c = CreateChart(ws)
    c.Location(xlc.xlLocationAsNewSheet, n)
    return ws.Parent.Charts(n)

def linest(s):
    c = ws.Cells(1,1).End(xlc.xlToRight).Offset(1, 3)
    c.Value = "kLa"
    c2 = c.Offset(2,1)
    lin = "=Index(Linest(%s,%s), %%d)" % (s.yr.Address, s.xr.Address)
    c2.Value = lin % 1
    c2.Offset(1,2).Value = lin % 2
    return c2

def mkcmp():
    global nwb, nws, cmpc
    nwb = xl.Workbooks.Add()
    nws = nwb.Worksheets(1)
    cmpc = nch(nws, "Comparison")
    c1 = nws.Cells.Range("B2")
    c1.Value = "Name"
    c1.Offset(1,2).Value = "kLa"
    nws.Cells.Range(c1.Offset(0, 1), c1.Offset(0, 2)).Merge()
    c1.Offset(0, 1).Value = "Compiled kLA Data"
    mgs = [[i/10 for i in range(1,6)]]
    o2s = [(i,) for i in (20,40,60,80,100)]
    
    c2 = nws.Cells.Range("P3:T3")
    c3 = nws.Cells.Range("O4:O8")
    c2.Value = mgs
    c3.Value = o2s
        
    
def lincmp2(name, kla):
    c1 = nws.Cells.Range("B3")
    if not c1.Value:
        c = c1
    elif c1.Value and not c1.Offset(2,1).Value:
        c = c1.Offset(2,1)
    else:
        c = c1.End(xlc.xlDown).Offset(2,1)
    c.Value = name
    c.Offset(1,2).Value = kla
    
def lincmp(name, kla, mg, o2):
    row = o2 // 20
    col = int(mg*10)
    c1 = nws.Cells.Range("P4").Offset(row, col)
    c1.Value = kla
    lincmp2(name, kla)
    
def gfp(f): return os.path.abspath(f)

def check_no_flow(name, nmax=2):
    down = column_data(name)
    ndown = len([d for d in down if d > 0])
    if ndown > nmax:
        print("ERROR: Check %s for %r"%(name, f))
        
def check_flow(name, exp, nmax=5):
    flow = column_data(name)
    nflow = len([d for d in flow if round(d, 0) != round(exp, 0)])
    if nflow > nmax:
        print("ERROR: Check %s"%(name))

    
def sorteddir():
    return sorted((f for f in os.listdir() if os.path.isfile(f)))

def trendclr(c):
    ns = c.SeriesCollection().Count
    l = c.Legend.LegendEntries
    for i in range(2*ns, ns, -1):
        l(i).Delete()
        
def mktrend(c):
    AddTrendlines(c)
    trendclr(c)
    
    
def anylf(wb,ws,name,min_start=0,rampdown=False, mg=0, o2=0):
    e1, e2 = add_af1("Elapsed", "Elapsed.hr", "=%s/3600")
    ws.Cells.Range(e1, e2).NumberFormat = "0.000"
    
    # calculate time offset
    tdat = column_data('Elapsed.hr')
    offs = next((i for i, t in enumerate(tdat) if t > min_start), 0)
    
    if rampdown:
        cname = "Ramp Down"
        target = "do.pv"
        yax = "DO PV (%)"
    else:
        cname = "Ramp Up"
        target = "-do.ln(pv)"
        yax = "-LN(100-DOPV)"
        add_af1("do.pv", target, "=-ln(100-%s)")

    c = make_chart(target, "%s %s"%(name, cname), "Time(hr)", 
                   yax, "Elapsed.hr", uws=ws, begin_offset=offs)
    c.series[0].set_name(name)
    mktrend(c.chart)

    if rampdown:
        # by the time 4 values are recorded, maingas should be set to the test's sp....
        check_flow("ph.outputDown", 0.1/(column_data('maingas.man')[3])*100)
        kla=0
    else:
        check_no_flow("ph.outputDown")
        check_no_flow("do.outputDown")
        check_flow("do.outputUp", o2)  # flow is always 100 for Sam's tests
        
        # MFCs
        mfcc = [
            "MFCs.Air",
            "MFCs.O2",
            "MFCs.N2",
            "MFCs.CO2"
        ]
        make_chart(mfcc, "MFC Flow", "Time(s)", "Flow Rate (LPM)")
        make_chart("do.pv", "DO PV", "Time(s)", "DO PV (%)")
        
        lc = linest(c.series[0])
        kla = lc.Value
        
    return c, kla
    
def analyze4(files, min_start=0.0, cmp_name="compiled"):
    # globals make debugging easy
    global nwb, nws, nc, wb, ws, e1, e2, pc, c, cmpc
    mkcmp()
    xl.DisplayAlerts=False
    for f in files:
        _, _, _, ramp, _, mg, _, o2, _ = f.split()
        mg = float(mg)
        o2 = int(o2[:-1])
        name = "%.1f LPM %d%% O2"%(mg, o2)
        rampdown = ramp == 'rampdown'
        if rampdown: continue
        print("Analyzing %r"%f)
        wb = xl.Workbooks.Open(gfp(f))
        ws = wb.Worksheets(1)
        
        c, kla = anylf(wb,ws,name,min_start,rampdown, mg, o2)
             
        if not rampdown:
            c.copyto(cmpc)
            lincmp(name, kla, mg, o2)
            
        wb.SaveAs(gfp(".\\analyzed2\\"+f.replace(".csv", ".xlsx")), FileFormat=xlc.xlOpenXMLWorkbook)
        wb.Close(True)
        
    nwb.SaveAs(gfp(".\\analyzed2\\%s.xlsx"%cmp_name))
    print("Finished analysis!")

In [26]:
fp = 'C:\\Users\\natha\\Documents\\test\\do dip tube many maingas'
xl = Excel()
os.chdir(fp)
os.makedirs("analyzed2", exist_ok=True)
files = sorteddir()
with screen_lock(xl):
    for wb in xl.Workbooks: wb.Close(False)
    analyze4(files, 0.2, "cmp_prelim2")

Analyzing 'DO micro test rampup mg 0.1 o2 100% 170804.csv'
Analyzing 'DO micro test rampup mg 0.1 o2 20% 170804.csv'
Analyzing 'DO micro test rampup mg 0.1 o2 40% 170804.csv'
Analyzing 'DO micro test rampup mg 0.1 o2 60% 170804.csv'
Analyzing 'DO micro test rampup mg 0.1 o2 80% 170804.csv'
Analyzing 'DO micro test rampup mg 0.2 o2 100% 170805.csv'
Analyzing 'DO micro test rampup mg 0.2 o2 20% 170804.csv'
Analyzing 'DO micro test rampup mg 0.2 o2 40% 170804.csv'
Analyzing 'DO micro test rampup mg 0.2 o2 60% 170804.csv'
Analyzing 'DO micro test rampup mg 0.2 o2 80% 170804.csv'
Analyzing 'DO micro test rampup mg 0.3 o2 100% 170805.csv'
Analyzing 'DO micro test rampup mg 0.3 o2 20% 170805.csv'
Analyzing 'DO micro test rampup mg 0.3 o2 40% 170805.csv'
Analyzing 'DO micro test rampup mg 0.3 o2 60% 170805.csv'
Analyzing 'DO micro test rampup mg 0.3 o2 80% 170805.csv'
Analyzing 'DO micro test rampup mg 0.4 o2 100% 170805.csv'
Analyzing 'DO micro test rampup mg 0.4 o2 20% 170805.csv'
Analyzing 